In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.api import OLS

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)

In [ ]:
# Load the data
all_data = pd.read_csv('MSF_1996_2023.csv', low_memory=False)

# Convert date to datetime
all_data['date'] = pd.to_datetime(all_data['date'])

# Sort the dataframe by date and PERMNO
all_data

In [ ]:
all_data = all_data.sort_values(['date', 'PERMNO'])

In [ ]:
def map_industry(siccd):
    if 1 <= siccd <= 999:
        return "Agriculture, Forestry and Fishing"
    elif 1000 <= siccd <= 1499:
        return "Mining"
    elif 1500 <= siccd <= 1799:
        return "Construction"
    elif 2000 <= siccd <= 3999:
        return "Manufacturing"
    elif 4000 <= siccd <= 4999:
        return "Transportation and other Utilities"
    elif 5000 <= siccd <= 5199:
        return "Wholesale Trade"
    elif 5200 <= siccd <= 5999:
        return "Retail Trade"
    elif 6000 <= siccd <= 6799:
        return "Finance, Insurance and Real Estate"
    elif 7000 <= siccd <= 8999:
        return "Services"
    else:
        return "Public Administration"

In [ ]:
industry_labels = [
    "Agriculture, Forestry and Fishing",
    "Mining",
    "Construction",
    "Manufacturing",
    "Transportation and other Utilities",
    "Wholesale Trade",
    "Retail Trade",
    "Finance, Insurance and Real Estate",
    "Services",
    "Public Administration"
]

In [ ]:
all_data['SICCDINT'] = pd.to_numeric(all_data['SICCD'], errors='coerce')
all_data = all_data.dropna(subset=['SICCDINT'])
all_data['industry'] = all_data['SICCDINT'].apply(map_industry)

In [ ]:
def sample_unique_companies(group):
    unique_companies = group.drop_duplicates(subset='PERMNO')
    if len(unique_companies) > 10:
        return unique_companies.sample(10)
    else:
        return unique_companies

def select_sample(df, year):
    year_data = df[df['date'].dt.year == year]
    return year_data.groupby('industry').apply(sample_unique_companies).reset_index(drop=True)

In [ ]:
all_data['RET'] = pd.to_numeric(all_data['RET'], errors='coerce')
all_data['vwretd'] = pd.to_numeric(all_data['vwretd'], errors='coerce')
all_data = all_data.dropna(subset=['RET', 'vwretd'])

In [ ]:
samples = [select_sample(all_data, year) for year in range(1996, 2024)]

In [ ]:
final_df = all_data.copy(deep=True)

In [ ]:
def estimate_beta(returns, market_returns):
    X = sm.add_constant(market_returns)
    model = OLS(returns, X).fit()
    beta = model.params[1]  # Beta is the coefficient of market_returns
    alpha = model.params[0]  # Alpha is the intercept
    residuals = model.resid
    residual_var = residuals.var()
    var_market_returns = market_returns.var()
    var_returns = returns.var()
    
    return beta, alpha, residual_var, var_market_returns, var_returns

In [ ]:
beta_df = pd.DataFrame(columns=['PERMCO', 'Year', 'Window', 'Beta', 'Alpha', 'Residual Var', 'Market Var', 'Stock Var', 'Industry'])

In [ ]:
def calculate_betas_for_sample(full_data, sample_firms, beta_df1, year, window):
    sample_permnos = sample_firms['PERMNO'].unique()
    full_data = full_data.sort_values(['PERMNO', 'date'])
    data_for_beta = full_data[full_data['PERMNO'].isin(sample_permnos)]
    for firm in sample_permnos:
        firm_data = data_for_beta[(data_for_beta['PERMNO'] == firm) & (data_for_beta['date'].dt.year <= year) & (data_for_beta['date'].dt.year > (year - (window)/12))]
        if(firm_data['RET'].shape[0] < 10):
            continue
        beta, alpha, residual_var, var_market_returns, var_returns = estimate_beta(firm_data['RET'], firm_data['vwretd'])
        industry_value = firm_data['industry'].iloc[0] 

        new_row = {
            'PERMCO': firm,
            'Year': year,
            'Window': window,
            'Beta': beta,
            'Alpha': alpha,
            'Residual Var': residual_var,
            'Market Var': var_market_returns,
            'Stock Var': var_returns,
            'Industry': industry_value
        }

        beta_df1 = beta_df1.append(new_row, ignore_index=True)
    return beta_df1

In [ ]:
# Calculate betas for each sample
year = 1996
beta_df = pd.DataFrame(columns=['PERMCO', 'Year', 'Window', 'Beta', 'Alpha', 'Residual Var', 'Market Var', 'Stock Var', 'Industry'])
for sample in samples:
    beta_df = calculate_betas_for_sample(final_df, sample, beta_df, year , 12)
    beta_df = calculate_betas_for_sample(final_df, sample, beta_df, year , 24)
    beta_df = calculate_betas_for_sample(final_df, sample, beta_df, year , 36)
    year = year + 1

In [ ]:
# sample_permnos = samples[0]['PERMNO'].unique()
# final_df = final_df.sort_values(['PERMNO', 'date'])
# data_for_beta = final_df[final_df['PERMNO'].isin(sample_permnos)]
# data_for_beta['PERMNO'].unique()


import pandas as pd
from scipy.stats import skew, kurtosis

def descriptive_stats(group):
    return pd.Series({
        'N': group.count(),
        'mean': group.mean(),
        'std_dev': group.std(),
        'min': group.min(),
        'max': group.max(),
        '1%': group.quantile(0.01),
        '5%': group.quantile(0.05),
        '25%': group.quantile(0.25),
        '50%': group.median(),
        '75%': group.quantile(0.75),
        '95%': group.quantile(0.95),
        '99%': group.quantile(0.99),
        'skewness': skew(group, bias=False),
        'kurtosis': kurtosis(group, bias=False)
    })

In [ ]:
# Group by industry and apply the descriptive_stats function to 'Beta' column
result = beta_df.groupby('Industry')['Beta'].apply(descriptive_stats).unstack()

In [ ]:
year_wise_result = beta_df.groupby(['Industry', 'Year'])['Beta'].apply(descriptive_stats).unstack()

<h1>
    Descriptive Stats Industry Wise
</h1>

In [ ]:
from IPython.display import display, HTML

# Convert the DataFrame to an HTML table with scrollable output
display(HTML(result.to_html(max_rows=500, max_cols=20)))

<h1>
    Descriptive Stats Industry-Year Wise
</h1>

In [ ]:
display(HTML(year_wise_result.to_html(max_rows=500, max_cols=20)))

<h1>
    Mean Betas Graph across industries
</h1>

In [ ]:
import matplotlib.pyplot as plt
# Group by 'industry' and 'year', then compute mean and standard deviation
agg_stats = beta_df.groupby(['Industry', 'Year'])['Beta'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
agg_stats.columns = ['Industry', 'Year', 'mean_beta', 'std_beta']

# Set up the plotting area
plt.figure(figsize=(24, 8))

# Get unique industries
industries = agg_stats['Industry'].unique()

# Plot each industry's mean and standard deviation over time
for industry in industries:
    industry_data = agg_stats[agg_stats['Industry'] == industry]
    
    plt.plot(industry_data['Year'], industry_data['mean_beta'], label=f'{industry} Mean', linestyle='-', marker='o')
#     plt.fill_between(industry_data['Year'],
#                      industry_data['mean_beta'] - industry_data['std_beta'],
#                      industry_data['mean_beta'] + industry_data['std_beta'],
#                      alpha=0.3)

# Add labels and title
plt.xlabel('Year')
plt.ylabel('Beta')
plt.title('Mean of Betas by Industry (1996-2023)')
plt.legend(loc='best')
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# code for plotting here
import plotly.express as px
def plotTimeSeries(df, label):
#     try:
        fig = px.line(df, x=df.index, y = df['mean_beta'],  title="Plot for " + label)
        fig.update_xaxes(nticks=30)
        fig.update_layout(
            height=600,
            width=800
        )
        fig.show()
#     except:
#         print("Error plotting " + label)

<h1>
    Std Deviation of Betas Across industries
</h1>

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for industry in industries:
    industry_data = agg_stats[agg_stats['Industry'] == industry]
    fig.add_trace(go.Scatter(
        x=industry_data['Year'],
        y=industry_data['std_beta'],
        mode='lines+markers',
        name=f'{industry} Std',
        line=dict(width=1),
        marker=dict(size=3)
    ))

fig.update_layout(
    title='Standard Deviation of Betas by Industry (1996-2023)',
    xaxis_title='Year',
    yaxis_title='Beta',
    legend_title='Industry',
    width=1200,
)
fig.update_xaxes(nticks=28)

# Show the plot
fig.show()

<h4>

**Key observations and trends::** 

- Volatility: Most industries show significant volatility in their beta values over time, with frequent ups and downs rather than steady trends
- Range: Beta values generally range between 0.5 and 2.5 for most industries across the period, especially after 2004
- Significant drop in Beta seen in Finance and Banking industry in 2008 likely owing to the financial crisis.
- High Beta is observed in Construction and Mining sectors. THis likely means that these industries tend to perform better than the market overall as they are a more stable business. 
- Mining shows huge spike in 2008 and 2020 both in Beta and Volatility of Beta. This means irregular impact in the markets for the mining industry during the financial crisis and COVID.
-  In the most recent years during 2017 and 2021, there's increased volatility across many sectors, possibly reflecting the economic uncertainties related to the COVID-19 pandemic. Reason for 2017 is still inconclusive.
</h4>

In [ ]:
beta_df['IVOL'] = np.sqrt(beta_df['Residual Var'])
beta_df['SVOl'] = np.sqrt((beta_df['Beta'] ** 2) * beta_df['Market Var'])
beta_df['TVOL'] = beta_df['SVOl'] + beta_df['IVOL']

In [ ]:
agg_vol_stats = beta_df.groupby(['Industry', 'Year'])['TVOL','SVOl','IVOL'].agg(['mean']).reset_index()
agg_vol_stats.columns = ['Industry', 'Year', 'TVOL', 'SVOL', 'IVOL']

In [ ]:
import plotly.graph_objects as go

def plotVolatility(agg_stats, industries):
    # Loop through each industry to create a separate plot
    for industry in industries:
        # Filter data for the current industry
        industry_data = agg_stats[agg_stats['Industry'] == industry]
        
        # Ensure 'Year' is numeric and sorted
#         industry_data['Year'] = pd.to_numeric(industry_data['Year'], errors='coerce')
        industry_data = industry_data.sort_values(by='Year')

        # Initialize a figure
        fig = go.Figure()

        # Plot Total Volatility (TVOL)
        fig.add_trace(go.Scatter(
            x=industry_data['Year'],
            y=industry_data['TVOL'],
            mode='lines+markers',
            name=f'{industry} TVOL',
            line=dict(width=1),
            marker=dict(size=4)
        ))

        # Plot Systematic Volatility (SVOL)
        fig.add_trace(go.Scatter(
            x=industry_data['Year'],
            y=industry_data['SVOL'],
            mode='lines+markers',
            name=f'{industry} SVOL',
            line=dict(width=1),
            marker=dict(size=4)
        ))

        # Plot Idiosyncratic Volatility (IVOL)
        fig.add_trace(go.Scatter(
            x=industry_data['Year'],
            y=industry_data['IVOL'],
            mode='lines+markers',
            name=f'{industry} IVOL',
            line=dict(width=1),
            marker=dict(size=4)
        ))

        # Update layout with title and axis labels
        fig.update_layout(
            title=f'Volatility Metrics for {industry} (1996-2023)',
            xaxis_title='Year',
            yaxis_title='Volatility',
            legend_title='Volatility Type',
            xaxis=dict(tickmode='linear', tick0=1996, dtick=1),  # Ensure yearly ticks on x-axis
            width=1000,  # Increase figure width for better readability
            height=600
        )
        
        # Display the figure
        fig.show()

# Example: Assuming you have the 'agg_stats' DataFrame and a list of industries
# plotVolatility(agg_stats, industries)


<h1>
    Volatility Plots Industry wise Graph for TVOL,IVOL,SVOL
</h1>

In [ ]:
plotVolatility(agg_vol_stats, industries)

<h2>

### Observation

- IVOL major contributor to the TVOL. Most volatility comes from residual volatility of linear regression. This makes sense and sustematic market volatility is less as overall market is more stable than individual stocks. 
- TVOL, IVOL and SVOL  are dependent and correlated. They all follow the same distribution of trends ie it is not seen that the volatility trends differ from one another (ie. not a single graph with unnatural trends like SVOL increases but TVOL decreases etc.). 
- Services industry seen to have relativele stable volatility since 2005
- Highest Volatility seen in mining
- Sharp spikes in TVOL, IVOL and SVOL seen during 2008-2010 period owing to the financial crisis
- Sharp spikes in TVOL, IVOL and SVOL seen during 2020-2021 period owing to the COVID pandemic

</h2>

In [ ]:
betas_portfolio = beta_df.copy(deep=True)

In [ ]:
yearly_df = final_df.groupby(pd.Grouper(key='date', freq='Y')).agg({'vwretd': 'mean'}).reset_index()
yearly_df['date'] = yearly_df['date'].dt.year
yearly_df.rename(columns={'date': 'Year'}, inplace=True)

In [ ]:
betas_portfolio = betas_portfolio.merge(yearly_df, on='Year', how='left')

In [ ]:
betas_portfolio['returns_excess'] = (betas_portfolio['Beta']) * betas_portfolio['vwretd'] + betas_portfolio['Alpha']

In [ ]:
def compute_market_cap_percentage(df):
#     yearly_vol = df.groupby(pd.Grouper(key='date', freq='Y')).agg({'vwretd': 'mean'}).reset_index()
    df['MarketCap'] = np.abs(df['SHROUT'] * df['PRC'])
    yearly_volume = df.groupby([pd.Grouper(key='date', freq='Y'), 'PERMNO'])['MarketCap'].sum().reset_index()
    yearly_volume.rename(columns={'date': 'Year'}, inplace=True)
    total_volume = yearly_volume.groupby('Year')['MarketCap'].sum().reset_index()
    total_volume.columns = ['Year', 'TotalCap']
    merged_data = pd.merge(yearly_volume, total_volume, on='Year')
    merged_data['MarketCapWeight'] = (merged_data['MarketCap'] / merged_data['TotalCap']) * 100
    merged_data['Year'] = merged_data['Year'].dt.year
    return merged_data[['PERMNO', 'Year', 'MarketCapWeight']]

In [ ]:
market_cap = compute_market_cap_percentage(final_df)

In [ ]:
market_cap.rename(columns={'PERMNO': 'PERMCO'}, inplace=True)

In [ ]:
beta_portfolio_valueweighted = pd.merge(betas_portfolio, market_cap, on=['Year', 'PERMCO'], how='inner')

In [ ]:
beta_portfolio_valueweighted['return_weighted'] = beta_portfolio_valueweighted['returns_excess']*beta_portfolio_valueweighted['MarketCapWeight']/100

In [ ]:
beta_portfolio_valueweighted

In [ ]:
def createPortfolio(df):
    df_sorted = df.sort_values(by='Beta')
    df_sorted['quintile'] = pd.qcut(df_sorted['Beta'], 5)
    avg_beta = df_sorted.groupby('quintile')['Beta'].mean()
    avg_excess_return = df_sorted.groupby('quintile')['returns_excess'].mean()
    print("Average Beta by Quintile:\n", avg_beta)
    print ("\n")
    
    print("Equal weighted Portfolio distributions:")
    difference_high_low_beta = avg_excess_return.loc[5] - avg_excess_return.loc[1]
    print("Average Excess Return by Quintile:\n", avg_excess_return)
    print("Difference in Excess Return (High Beta 5 - Low Beta 1):", difference_high_low_beta)
    print ("\n")
    
    print("Value weighted Portfolio distributions:")
    avg_excess_return_vw = df_sorted.groupby('quintile')['return_weighted'].mean()
    difference_high_low_beta_vw = avg_excess_return_vw.loc[5] - avg_excess_return_vw.loc[1]
    print("Average Excess Return by Quintile:\n", avg_excess_return_vw)
    print("Difference in Excess Return Value Weighted (High Beta 5 - Low Beta 1):", difference_high_low_beta_vw)
    

In [ ]:
def createPortfolioIVol(df):
    df_sorted = df.sort_values(by='IVOL')
    df_sorted['quintile'] = pd.qcut(df_sorted['IVOL'], 5)
    avg_beta1 = df_sorted.groupby('quintile')['Beta'].mean()
    avg_excess_return1 = df_sorted.groupby('quintile')['returns_excess'].mean()
    print("Average Beta by Quintile:\n", avg_beta1)
    print ("\n")
    
    print("Equal weighted Portfolio distributions:")
    print("Average Excess Return by Quintile:\n", avg_excess_return1)
#     difference_high_low_beta = avg_excess_return[5] - avg_excess_return[1]
#     print("Difference in Excess Return (High IVOL 5 - Low IVOL 1):", difference_high_low_beta)
    print ("\n")
    
    print("Value weighted Portfolio distributions:")
    avg_excess_return_vw = df_sorted.groupby('quintile')['return_weighted'].mean()
#     difference_high_low_beta_vw = avg_excess_return_vw.loc[5] - avg_excess_return_vw.loc[1]
    print("Average Excess Return by Quintile:\n", avg_excess_return_vw)
#     print("Difference in Excess Return Value Weighted (High IVOL 5 - Low IVOL 1):", difference_high_low_beta_vw)

<h1>
    Portfolio Quintile Information (Sorted by Beta)
</h1>

In [ ]:
createPortfolio(beta_portfolio_valueweighted)

<h1>
    Portfolio Quintile Information (Sorted by IVOL)
</h1>

In [ ]:
createPortfolioIVol(beta_portfolio_valueweighted)

In [ ]:
<h4>
References: Some portion of the Code for generating graphs used CHATGPT
</h4>